In [4]:
import numpy as np  # Импортируем библиотеку NumPy для работы с массивами и матрицей
from sklearn.datasets import fetch_openml  # Импортируем модуль для загрузки набора данных
from sklearn.model_selection import train_test_split  # Модуль для разделения данных на тренировочные и тестовые
from sklearn.metrics import accuracy_score  # Модуль для измерения точности модели

class Perceptron:
    """
    Класс, представляющий однослойный перцептрон.
    """
    def __init__(self, learning_rate=0.25, epochs=25):
        """
        Конструктор класса, инициализирующий параметры модели.

        :param learning_rate: Скорость обучения
        :param epochs: Количество эпох обучения
        """
        self.learning_rate = learning_rate  # Сохраняем значение скорости обучения
        self.epochs = epochs  # Сохраняем количество эпох обучения
        self.weights = None  # Начальные веса будут установлены позже
        self.bias = None  # Смещение будет установлено позже

    def sigmoid(self, x):
        """
        Реализация сигмоидальной функции активации.

        :param x: Входной вектор или скаляр
        :return: Значение сигмоиды от x
        """
        return 1 / (1 + np.exp(-x))  # Формула сигмоидной функции

    def predict(self, X):
        """
        Функция для предсказания классов на основе текущих весов и смещений.

        :param X: Матрица входных данных (признаки)
        :return: Вектор предсказанных классов (0 или 1)
        """
        linear_output = np.dot(X, self.weights) + self.bias  # Линейный выход = сумма произведений весов и признаков плюс смещение
        probabilities = self.sigmoid(linear_output)  # Применяем сигмоидальную активацию к линейному выходу
        return (probabilities >= 0.5).astype(int)  # Возвращаем классы: 1, если вероятность больше 0.5, иначе 0

    def fit(self, X_train, y_train):
        """
        Обучающая функция модели с применением алгоритма положительного и отрицательного подкрепления.

        :param X_train: Тренировочная матрица признаков
        :param y_train: Тренировочные метки классов
        """
        n_samples, n_features = X_train.shape  # Определяем количество образцов и признаков

        # Инициализация весов и смещения случайными значениями
        self.weights = np.random.normal(size=n_features)  # Инициализация весов нормальным распределением
        self.bias = np.random.normal()  # Инициализация смещения случайным числом

        for epoch in range(self.epochs):  # Проходимся по заданному количеству эпох
            print(f'Эпоха {epoch + 1}/{self.epochs}:')  # Выводим номер текущей эпохи

            # Перемешиваем данные перед каждым проходом
            idx = np.arange(n_samples)  # Индексируем образцы
            np.random.shuffle(idx)  # Перемешиваем индексы
            X_shuffled = X_train[idx]  # Смешанный набор данных
            y_shuffled = y_train[idx]  # Соответствующие смешанные метки

            for xi, yi in zip(X_shuffled, y_shuffled):  # Для каждого образца в перемешанном наборе
                output = self.sigmoid(np.dot(xi, self.weights) + self.bias)  # Вычисляем выход нейрона
                error = yi - output  # Ошибка (целевое значение минус предсказанное)

                # Если ошибка положительная (персептрон недооценил значение), применяем положительное подкрепление
                if error > 0:
                    # Усиливаем веса и смещение пропорционально величине ошибки
                    self.weights += self.learning_rate * error * xi
                    self.bias += self.learning_rate * error
                elif error < 0:
                    # Если ошибка отрицательная (персептрон переоценил значение), применяем отрицательное подкрепление
                    # Ослабляем веса и смещение пропорционально величине ошибки
                    self.weights -= self.learning_rate * abs(error) * xi
                    self.bias -= self.learning_rate * abs(error)
                else:
                    # Если ошибка равна нулю, ничего не меняем
                    pass

# Загружаем датасет MNIST
mnist = fetch_openml('mnist_784', version=1)  # Загружаем данные MNIST
X = mnist['data']  # Извлекаем данные изображений (векторизованные пиксели)
y = mnist['target'].astype(np.int8)  # Извлекаем метки классов (цифры от 0 до 9)

# Преобразовываем данные в Numpy-массивы
X = X.to_numpy()  # Преобразуем данные изображений в Numpy-массив
y = y.to_numpy()  # Преобразуем метки в Numpy-массив

# Преобразуем метки в бинарные (цифры 0 против всех остальных)
y_binary = (y != '0').astype(int)  # 0 заменяется на 0, другие цифры на 1

# Делиим данные на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42)  # 20% данных отводится на тестовый набор

# Нормализуем данные (масштабируем пиксельные значения от 0 до 1)
X_train = X_train / 255.0  # Делим каждый пиксель на 255 (максимальное значение)
X_test = X_test / 255.0  # То же самое делаем для тестового набора

# Создаем и обучаем модель
perceptron = Perceptron()  # Создаем экземпляр класса Perceptron
perceptron.fit(X_train, y_train)  # Запускаем обучение модели

# Прогнозируем результаты на тестовых данных
predictions = perceptron.predict(X_test)  # Получаем предсказанные классы

# Оцениваем точность модели
accuracy = accuracy_score(y_test, predictions)  # Сравниваем реальные метки с предсказаниями
print(f'Accuracy: {accuracy:.4f}')  # Выводим точность модели с точностью до четырех знаков

Эпоха 1/25:
Эпоха 2/25:
Эпоха 3/25:
Эпоха 4/25:
Эпоха 5/25:
Эпоха 6/25:
Эпоха 7/25:
Эпоха 8/25:
Эпоха 9/25:
Эпоха 10/25:
Эпоха 11/25:
Эпоха 12/25:
Эпоха 13/25:
Эпоха 14/25:
Эпоха 15/25:
Эпоха 16/25:
Эпоха 17/25:
Эпоха 18/25:
Эпоха 19/25:
Эпоха 20/25:
Эпоха 21/25:
Эпоха 22/25:
Эпоха 23/25:
Эпоха 24/25:
Эпоха 25/25:
Accuracy: 1.0000
